# Energy calibration

In [ ]:
using Revise
push!(LOAD_PATH, "$(homedir())/Development/")
using JSort
using Plots
using PyCall

function plotmat(matrix)
    G = convert(Array{Float64, 2}, matrix)
    log10transform!(G)
    pl = heatmap(G', c=:viridis, yticks=([1:length(chns);], chns))
end

function plotstrip(strips)
    G = hcat(strips...)
    G = convert(Array{Float64, 2}, G)
    log10transform!(G)
    pl = heatmap(G', c=:viridis)
end

function log10transform!(G)
    G[G .> 0] .= log10.(G[G .> 0])
    G[G .≤ 0] .= NaN
end

function log10transform(V)
    G = V[1:end]
    G[G .> 0] .= log10.(G[G .> 0])
    G[G .≤ 0] .= NaN
    G
end

In [ ]:
# Load the preprocessed data
events = loadlabr("sirius");
parameters = Parameters("si28.yaml");

# Sort the datafile
ge, gt = makelabr(events, parameters);
mat = copy(ge.matrix);

# Find the good channels
chns = goodchannels(mat);
G = mat[:, chns];
M = [G[:, i] for i in 1:size(G, 2)];
pop!(M);

In [ ]:
plotstrip(M)

In [ ]:
signal = pyimport("scipy.signal")
smoother(x) = signal.savgol_filter(x, 51, 4)
coefficients, alignedspectra = alignspectra(M, lowregion=3:1400, highregion=1350:1800,
                                            highsmoother=smoother)
coefficients, alignedspectra = alignspectra(alignedspectra, lowregion=3:1400, 
                                            highregion=1350:1800, lowsearchwidth=10,
                                            highsearchwidth = 10, lownumregions=15,
                                            highsmoother=smoother)

plotstrip(alignedspectra)

In [ ]:
total = sum(alignedspectra)
plt = plot(xlim=(1500, 2000), ylim=(0, 100))
for spectrum in alignedspectra
    plt = plot!(spectrum)
end
display(plt)
plot(total, scale=:symlog)

In [ ]:
using DataFrames
import CSV

In [ ]:
df = DataFrame(total=total)
for i in eachindex(alignedspectra)
    df[!, Symbol(i)] = alignedspectra[i]
end